<a href="https://colab.research.google.com/github/sherrihoumadi/Data-science-design-manual-notebooks/blob/master/Copy_of_paint_images_ipynb_transfer_resnet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np 
import os, cv2, sys, shutil

from keras.models import Sequential,model_from_json
from keras.layers import Input, Dropout, Flatten, Convolution2D, MaxPooling2D, Dense, Activation
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import np_utils
from keras import applications
from keras.models import Model
from keras import optimizers
from keras.layers.normalization import BatchNormalization
from keras.metrics import categorical_accuracy
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping
from keras import backend as K
from random import sample 


Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount ('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
# Manually Created empty validation folder on drive. As of now you can move all of you images from "test" folder to respective folder in "train" directory
# Next block has small logic to randomly sample the images to be used as validation set.

base_dir = os.path.join("/content/gdrive/My Drive/", 'Paint_Pics')
np.random.seed(100)
train_dir = os.path.join(base_dir, 'train')
#validation_dir = os.path.join(base_dir, 'test') # you can move contents of this folder to train folder. No longer required to do it manually 
validation_dir=os.path.join(base_dir, 'validation') # This is new validation set. This will be created using following sampling logic


ROWS = 128
COLS = 128
CHANNELS = 3

classes=len(os.listdir(train_dir))
max_train_step=50

In [0]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    print (recall)
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    print (precision)
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    print (2*((precision*recall)/(precision+recall+K.epsilon())))
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
# following script randomly selects images from train folder per class and puts it in validation folder .
# You can play with val_split parameter below to change the split %age.
# [Errno 1] Operation not permitted: "This error will go away once you run this from your end."" 

val_split=0.15  #(You can play with this)

for path, subdirs, files in os.walk(train_dir):
    try:
        files=sample(files,int(len(files)*val_split))
        path1=os.path.join(validation_dir,os.path.basename(path))
        if os.path.basename(path) !='train':
            if not os.path.exists(path1):
                os.makedirs(path1)

        for name in files:  
            shutil.move(os.path.join(path, name), os.path.join(path1, name))
    except Exception as E:
        print (E)
        pass

In [0]:
# Using pretrained model ResNet50 with pretrained weights from Imagenet (Transfer learning )
# This base model will extract the featues. 

base_model = applications.ResNet50(weights='imagenet', include_top=False, input_shape=(ROWS, COLS,3))

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 1s 0us/step


In [0]:
# This is fully connected layer after the base model. This helps us optimize the Imagenet weights for our class requirements.

add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(128, activation='relu'))
add_model.add(Dropout(0.30))
add_model.add(Dense(64, activation='relu'))
add_model.add(Dropout(0.30))
add_model.add(Dense(classes, activation='softmax'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))
model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=1e-3),
              metrics=['accuracy'] )

model_json = model.to_json()

#Saving the model in json format.
with open(os.path.join(base_dir,"model.json"), "w") as json_file:
    json_file.write(model_json)

#model.summary()


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [0]:
batch_size = 32

# this is the augmentation configuration we will use for training

train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10, 
        width_shift_range=0.2,
        height_shift_range=0.1,
        zoom_range=0.20,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        train_dir,  # this is the target directory
        target_size=(ROWS, COLS),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical'
        
        )  # since we use binary_crossentropy loss, we need binary labels
        #save_to_dir='datagen'
# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(ROWS, COLS),
        batch_size=batch_size,
        class_mode='categorical')

filepath=os.path.join(base_dir,"weights-improvement.hdf5")
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
callbacks = [EarlyStopping(monitor='val_acc', patience=15),
             ModelCheckpoint(filepath=filepath, monitor='val_acc', save_best_only=True)
             ]

history=model.fit_generator(
        train_generator,
        steps_per_epoch=1000// batch_size,
        epochs=max_train_step,
        validation_data=validation_generator,
        validation_steps=1000// batch_size,callbacks=callbacks)


Found 911 images belonging to 4 classes.
Found 316 images belonging to 4 classes.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/50
12/31 [==========>...................] - ETA: 2:02 - loss: 1.7780 - acc: 0.3380

FileNotFoundError: ignored